In [ ]:
import torch as th

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
def make_policy_obs_action(seq_len, ntoken, d_out, diff_ele, device = 'cuda'):
    o1 = th.zeros([1, seq_len, ntoken], dtype=th.float, device=device)
    o2 = th.zeros_like(o1)
    o2[0, diff_ele] = 1

    a1 = th.zeros(1, seq_len, d_out, dtype=th.float, device=device)
    a2 = th.zeros_like(a1)
    a2[:, diff_ele:] = 1
    o = th.cat((o1, o2), dim=0)
    a = th.cat((a1, a2), dim=0)
    
    return o, a


In [ ]:
observations, actions = make_policy_obs_action(3, 2, 1, 1, 'cuda')

In [ ]:
from ActiveCritic.utils.pytorch_utils import make_partially_observed_seq
import numpy as np
import gym
import torch as th
seq_len = 3
d_out = 2
d_in = 3
acts_array_low = [0]*d_out
acts_array_high = [1]*d_out

obs = th.ones([2, 1, d_in], dtype=th.float, device='cuda')

In [ ]:
action_space = gym.spaces.box.Box(
    np.array(acts_array_low), np.array(acts_array_high), (d_out,), float)

pos = make_partially_observed_seq(obs=obs, acts=None, seq_len=seq_len, act_space=action_space)
expected_shape = list(obs.shape)
expected_shape[1] = seq_len
expected_shape[2] += action_space.shape[0]
assert list(pos.shape) == expected_shape, f'Output shape is not as expected. Expected: {expected_shape}, but got {pos.shape}'
assert th.all(pos[:,:,obs.shape[-1]:] == 0), 'No action input, but some action fields are not zero.'
assert th.equal(pos[:,:obs.shape[1], :obs.shape[2]], obs), 'Observation not correctly inserted into sequence.'

In [ ]:
current_obs_len = 2
obs = th.ones([2, current_obs_len, d_in], dtype=th.float, device='cuda')
acts = 2*th.ones([2, current_obs_len-1, d_out], dtype=th.float, device='cuda')
pos = make_partially_observed_seq(obs=obs, acts=acts, seq_len=seq_len, act_space=action_space)

In [ ]:
expected_shape = list(obs.shape)
expected_shape[1] = seq_len
expected_shape[2] += action_space.shape[0]
assert list(pos.shape) == expected_shape, f'Output shape is not as expected. Expected: {expected_shape}, but got {pos.shape}'
assert th.all(pos[:,acts.shape[1],obs.shape[-1]:] == 0), 'Action that werent input are displayed.'
assert th.equal(pos[:,:obs.shape[1], :obs.shape[2]], obs), 'Observation not correctly inserted into sequence.'
assert th.equal(pos[:,:acts.shape[1], obs.shape[2]:], acts), 'Not all actions have been inserted correctly.'

In [14]:
import gym
from ActiveCritic.utils.gym_utils import make_policy_dict
from ActiveCritic.metaworld.metaworld.envs import ALL_V2_ENVIRONMENTS_GOAL_OBSERVABLE
from gym.wrappers import TimeLimit
from stable_baselines3.common.vec_env import DummyVecEnv
from imitation.data.wrappers import RolloutInfoWrapper
from ActiveCritic.utils.gym_utils import new_epoch_reach
import numpy as np
import torch as th
policy_dict = make_policy_dict()

env_tag = 'reach'
max_episode_steps = 5
re = ALL_V2_ENVIRONMENTS_GOAL_OBSERVABLE[policy_dict[env_tag][1]]()
re._freeze_rand_vec = False
timelimit = TimeLimit(env=re, max_episode_steps=max_episode_steps)
dv1 = DummyVecEnv([lambda: RolloutInfoWrapper(timelimit)])

/home/hendrik/anaconda3/envs/tfTest/lib/python3.9/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [15]:
from ActiveCritic.utils.gym_utils import DummyExtractor
de = DummyExtractor()

obs1 = de.forward(dv1.reset())

In [16]:
new_epoch_reach(obs1, obs1)

False

In [24]:
obs2_np, _,_,_ = dv1.step(np.array([[0,1,0,0]]))

In [25]:
obs2 = de.forward(obs2_np)

In [29]:
th.equal(obs2, obs1)

False

In [30]:
new_epoch_reach(obs1, obs2)

False

In [31]:
new_obs = de.forward(dv1.reset())

In [32]:
new_epoch_reach(obs1, new_obs)

True